In [1]:
#!pip install pinecone-client --upgrade

from pinecone import 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:

!pip install openai
!pip install python-dotenv
# !pip3 install pinecone-client
# !pip install PyPDF2
import dotenv
import openai
from PyPDF2 import PdfReader
import re
import os
import json
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")
embed_model = "text-embedding-ada-002"
index_name = 'cstugpt-kb'


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment="asia-southeast1-gcp-free" 
)

ModuleNotFoundError: No module named 'pinecone'

In [4]:


# Now do stuff
if 'my_index' not in pc.list_indexes().names():
    pc.create_index(
        name='my_index',
        dimension=1536,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

# # initialize connection to pinecone (get API key at app.pinecone.io)
# pinecone.init(
#     api_key=pinecone_api_key,
#     environment="asia-southeast1-gcp-free" # The environment value could be found at Pinecone website, once created
# )
# index = pinecone.Index(index_name)





def mls_upsert(cstu_file, index_name, name_space, cstu_id, chunk_size, stride):
   # create a reader object
    print("Knowledge base file name:", cstu_file)
    reader = PdfReader(cstu_file)
    page_len = len(reader.pages)
    print("length of the knowledge base file:", page_len)
    doc = ""
    for i in range(page_len):
        doc += reader.pages[i].extract_text()
        print("page completed:", i)
    doc = doc.splitlines()
    print("The total lines: ", len(doc))

    #Connect to Pinecone index
    index = pinecone.Index(index_name)
    count = 0
    for i in range(0, len(doc), chunk_size):#The loop iterates over the document in steps of chunk_size
        #find begining and end of the chunk
        i_begin = max(0, i-stride)
        i_end = min(len(doc), i_begin+chunk_size)
        doc_chunk = doc[i_begin:i_end]
        print("-------------------------------------------------------------")
        print("The ", i//chunk_size + 1, " doc chunk text:", doc_chunk)
        texts = ""
        for x in doc_chunk:
            texts += x
        print("Texts:", texts)

        #Create embeddings of the chunk texts
        try:
            res = openai.Embedding.create(input=texts, engine=embed_model)
        except:
            done = False
            while not done:
                sleep(5)
                try:
                    res = openai.Embedding.create(input=texts, engine=embed_model)
                    done = True
                except:
                    pass
        embed = res['data'][0]['embedding']
        print("Embeds length:", len(embed))

        # Meta data preparation
        metadata = {
            "cstu_id": cstu_id + '_' + str(count),
            "text": texts
        }
        count += 1
        print("Upserted vector count is: ", count)
        print("==========================================================")

        #upsert to pinecone and corresponding namespace
        index.upsert(vectors=[(metadata["cstu_id"], embed, metadata)], namespace=name_space)

NameError: name 'pinecone' is not defined

In [ ]:
mls_upsert(r"cstugpt_kb.pdf", "cstugpt-kb", "cstu","cstu-kb", 8, 1)
#mls_upsert(r"cstugpt_qa.pdf", "cstugpt-kb", "cstu","cstu-qa", 3, 1)

index.describe_index_stats()